In [10]:
import os
import sys
from pathlib import Path

In [11]:
# Set up proper directory paths

project_path = Path().absolute().parent
src_path = project_path.joinpath("src")

# Ensure the paths are properly assigned
# If this assertion fails, change project_dir as needed to become the project directory
# If project_dir is correct, change the name in the assertion check
assert project_path.name == "project-athena", "Parent directory name assertion failed (check the path)"

In [12]:
# Add src_dir to module paths
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

In [13]:
import time

import numpy as np

In [14]:
from matplotlib import pyplot as plt

In [15]:
from attacks.attack import generate
from models.athena import ENSEMBLE_STRATEGY, Ensemble
from utils.data import subsampling
from utils.file import load_from_json, dump_to_json
from utils.metrics import error_rate, get_corrections
from utils.model import load_lenet, load_pool

In [16]:
# copied from tutorials/craft_adversarial_examples.py
def generate_ae(model, data, labels, attack_configs,
                eot=False,
                save=False, output_dir=None):
    """
    Generate adversarial examples
    :param model: WeakDefense. The targeted model.
    :param data: array. The benign samples to generate adversarial for.
    :param labels: array or list. The true labels.
    :param attack_configs: dictionary. Attacks and corresponding settings.
    :param save: boolean. True, if save the adversarial examples.
    :param output_dir: str or path. Location to save the adversarial examples.
        It cannot be None when save is True.
    :return:
    """
    img_rows, img_cols = data.shape[1], data.shape[2]
    num_attacks = attack_configs.get("num_attacks")
    data_loader = (data, labels)

    if len(labels.shape) > 1:
        labels = np.asarray([np.argmax(p) for p in labels])

    # generate attacks one by one
    for id in range(num_attacks):
        key = "configs{}".format(id)
        attack_args = attack_configs.get(key)
        
        attack_args["eot"] = eot
        data_adv = generate(model=model,
                            data_loader=data_loader,
                            attack_args=attack_args
                            )
        # predict the adversarial examples
        predictions = model.predict(data_adv)
        predictions = np.asarray([np.argmax(p) for p in predictions])

        err = error_rate(y_pred=predictions, y_true=labels)
        print(">>> error rate:", err)

        # plotting some examples
        num_plotting = min(data.shape[0], 2)
        for i in range(num_plotting):
            img = data_adv[i].reshape((img_rows, img_cols))
            plt.imshow(img, cmap='gray')
            title = '{}(EOT:{}): {}->{}'.format(attack_configs.get(key).get("description"),
                                                "ON" if eot else "OFF",
                                                labels[i],
                                                predictions[i]
                                                )
            plt.title(title)
            plt.show()
            plt.close()

        # save the adversarial example
        if save:
            if output_dir is None:
                raise ValueError("Cannot save images to a none path.")
            # save with a random name
            file = os.path.join(output_dir, "{}.npy".format(time.monotonic()))
            print("Save the adversarial examples to file [{}].".format(file))
            np.save(file, data_adv)

In [18]:
# Load data configs
file = project_path.joinpath("Task2/configs/data-mnist.json")
data_configs = load_from_json(file)

trans_configs = load_from_json(project_path.joinpath("Task2/configs/athena-mnist.json"))
model_configs = load_from_json(project_path.joinpath("Task2/configs/model-mnist.json"))

# In the context of the white-box threat model,
# we use the ensemble as adversary's target model.
pool, _ = load_pool(trans_configs=trans_configs,
                    model_configs=model_configs,
                    active_list=True,
                    wrap=True)

# create an AVEP ensemble from the WD pool
wds = list(pool.values())
target = Ensemble(classifiers=wds, strategy=ENSEMBLE_STRATEGY.AVEP.value)

# load the benign samples
data_file = os.path.join(data_configs.get('dir'), data_configs.get('bs_file'))
data_bs = np.load(data_file)
# load the corresponding true labels
label_file = os.path.join(data_configs.get('dir'), data_configs.get('label_file'))
labels = np.load(label_file)

attack_configs = load_from_json(project_path.joinpath("Task2/configs/attack-zk-mnist.json"))
output_path = project_path.joinpath("Task2/data")

generate_ae(model=target, data=data_bs,labels=labels, attack_configs=attack_configs,
            save=True, output_dir=output_path)

>>> Loading model [../models/cnn/model-mnist-cnn-morph_gradient.h5]...
>>> Loading model [../models/cnn/model-mnist-cnn-distort_x.h5]...
>>> Loading model [../models/cnn/model-mnist-cnn-noise_gaussian.h5]...
>>> Loading model [../models/cnn/model-mnist-cnn-noise_sNp.h5]...
>>> Loading model [../models/cnn/model-mnist-cnn-filter_gaussian.h5]...
>>> Loading model [../models/cnn/model-mnist-cnn-geo_iradon.h5]...
>>> Loading model [../models/cnn/model-mnist-cnn-seg_gradient.h5]...
>>> Loaded 7 models.
>>> Generating PGD_eps0.05(EOT:OFF) examples.


KeyboardInterrupt: 

In [ ]:

attack_configs = load_from_json(project_path.joinpath("Task2/configs/attack-zk-mnistEOTON.json"))

# Adaptive approach (with EOT)
# Compute the loss expectation over specific distribution.
# For an ensemble target, averaging the EOT of WDs'.
# let save=True and specify an output folder to save the generated AEs
generate_ae(model=target, data=data_bs, labels=labels, eot=True, attack_configs=attack_configs, save=True, output_dir=output_path)

# Evaluating the generated adverserial example on the vanilla Athena.

In [22]:
file = project_path.joinpath("Task2/configs/data-mnist.json")
data_configs = load_from_json(file)

img_rows, img_cols = data_bs.shape[1], data_bs.shape[2]

pred_bs = target.predict(data_bs)
corrections = get_corrections(y_pred=pred_bs, y_true=labels)

results = {}

ae_files = data_configs.get('task2_aes')

for file in ae_files:
    results[file] = {}
    ae_file = project_path.joinpath(data_configs['dir'], file)
    x_adversarial = np.load(ae_file)
    print(f">>> Running evaluations on [{ae_file}]")
    pred = target.predict(x_adversarial)
    err = error_rate(y_pred=pred, y_true=labels, correct_on_bs=corrections)
    results[file] = err
    
result_file = project_path.joinpath("Task2/results/ae_evaluation_results.json")
dump_to_json(results, result_file)
print(f">>> Evaluations on all ae_files dumped to [{result_file}]")

>>> Running evaluations on [D:\GitHub\project-athena\Task2\data\111822.859.npy]
>>> Running evaluations on [D:\GitHub\project-athena\Task2\data\112149.578.npy]
>>> Running evaluations on [D:\GitHub\project-athena\Task2\data\112584.421.npy]
>>> Running evaluations on [D:\GitHub\project-athena\Task2\data\112962.656.npy]
>>> Running evaluations on [D:\GitHub\project-athena\Task2\data\113415.828.npy]
>>> Running evaluations on [D:\GitHub\project-athena\Task2\data\113459.828.npy]
>>> Running evaluations on [D:\GitHub\project-athena\Task2\data\113511.093.npy]
>>> Running evaluations on [D:\GitHub\project-athena\Task2\data\113581.718.npy]
>>> Running evaluations on [D:\GitHub\project-athena\Task2\data\113654.921.npy]
>>> Running evaluations on [D:\GitHub\project-athena\Task2\data\113724.812.npy]
>>> Evaluations on all ae_files dumped to [D:\GitHub\project-athena\Task2\results\ae_evaluation_results.json]
